# Time Series Modeling

In [222]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from random import gauss as gs
import datetime
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm

## White Noise

The idea behind white noise is that it is truly random.

We don't want white noise to describe our model per se, but we *do* want it to describe our model *error*.

Can you explain these truisms?

In [194]:
# Let's make some white noise!

rands = []
for _ in range(1000):
    rands.append(gs(0, 1))
series = pd.Series(rands)

In [244]:
X = np.linspace(-10, 10, 1000)
plt.figure(figsize=(10, 7))
plt.plot(X, series);

What happens if we do a seasonal decomposition on this?

In [227]:
time_index = pd.to_datetime(series.index, unit='D')

In [230]:
series.index = time_index
decomp_white = sm.tsa.seasonal_decompose(series)

In [253]:
plt.subplot(411)
plt.plot(series.index, decomp_white.observed, label='Original')
plt.legend()
plt.subplot(412)
plt.plot(series.index, decomp_white.trend, label='Trend')
plt.legend()
plt.subplot(413)
plt.plot(series.index, decomp_white.seasonal, label='Seasonal')
plt.legend()
plt.subplot(414)
plt.plot(series.index, decomp_white.resid, label='Residual')
plt.legend()
plt.subplots_adjust(hspace=0.5);

## Football Data

In [289]:
fb = pd.read_csv('data/google-trends_football_us.csv').iloc[1:, :]
fb.columns = ['counts']

In [290]:
fb['counts'] = fb['counts'].replace('<1', '0')
fb['counts'] = fb['counts'].astype(int)

In [245]:
fb.head()

### Series as Both Predictor and Target?

Often, the phenomenon we want to capture with a time series is a dataset being correlated with *itself*.

Well, of course every dataset is perfectly correlated with itself. But what we're after now is the idea that a series is correlated with *earlier versions* of itself.

Consider the problem of trying to predict tomorrow's closing price for some stock on the market. One may consider lots of features, like what sort of company it is to which the stock belongs or whether that company has been in the news recently.

But it is very often the case that one of the most helpful predictors of tomorrow's price is *today's* price. And so we want to build a model where one of our predictors is an earlier version of our target.

In [246]:
fb.rolling(window=2).mean().head()

In [273]:
fb['roll_avg'] = fb.rolling(window=2).mean()

fb.corr()

In [301]:
lr = LinearRegression()

lr.fit(fb[['roll_avg']][1:], fb['counts'][1:])

In [280]:
plt.figure(figsize=(20, 4))
plt.plot(fb.index[1:25], fb['counts'][1:25], label='Data')
plt.plot(fb.index[1:25], lr.predict(fb[['roll_avg']][1:25]), label='Predicted')
plt.legend();

### Autocorrelation and Partial Autocorrelation Functions

Pandas and statsmodels offer autocorrelation (ACF) and partial autocorrelation (PACF) plotting tools. The idea here is to look at the correlation of a series with itself for some particular interval or *lag*. The key difference between the full and the partial autocorrelation functions is that the partial autocorrelation function ignores intervening intervals. For more, see [this post](https://machinelearningmastery.com/gentle-introduction-autocorrelation-partial-autocorrelation/).

In [281]:
pd.plotting.autocorrelation_plot(fb);

#### Partial Autocorrelation

In [318]:
from statsmodels.graphics.tsaplots import plot_pacf
from matplotlib.pylab import rcParams

rcParams['figure.figsize'] = 14, 5

plot_pacf(fb, lags=20, alpha=0.01);

## ARMA Modeling

'AR' is for "Auto-Regressive": The prediction for today will be a function of the value for previous days.

The number of lag periods we want to include will be a parameter in the statsmodels model object ("p"). Looking at the PACF can help us decide on an appropriate p.

'MA' is for "Moving Average": The prediction for today will be a function of the rolling mean.

The number of average terms we want to include will be a parameter in the statsmodels model object ("q"). Looking at the ACF can help us decide on an appropriate q.

For some technical details, see [this page](https://en.wikipedia.org/wiki/Autoregressive%E2%80%93moving-average_model#Choosing_p_and_q).

In [291]:
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.arima_model import ARMA

### Stationarity and the Dickey-Fuller Test

ARMA models assume that the time series is *stationary*, which means that its statistical properties are not a (meaningful) function of time.

It may seem counterintuitive that, for modeling purposes, we want our time series not to be a function of time! But the basic idea is the familiar one that we want our datapoints to be mutually *independent*. For more on this topic, see [here](https://stats.stackexchange.com/questions/19715/why-does-a-time-series-have-to-be-stationary).

One way of testing for stationarity is to use the Dickey-Fuller Test. The statsmodels version returns the test statistic and a p-value, relative to the null hypothesis that the series in question is NOT stationary. For more, see [this Wikipedia page](https://en.wikipedia.org/wiki/Augmented_Dickey%E2%80%93Fuller_test).

In [283]:
# Presumably, our football series is not stationary. Let's check.

adfuller(fb['counts'], autolag=None)

But let's check the stationarity of the *differences* of our data:

In [ ]:
fb['counts'].head()

In [284]:
fb['counts'].diff().head()

In [285]:
adfuller(fb['counts'].diff()[1:], autolag=None)

### Building the Model

In [342]:
p=2
q=0
ar = ARMA(fb.diff().values[2:], (p, q)).fit()

In [346]:
ar.summary()

#### `np.cumsum()`
Let's use `np.cumsum()` to add up our predictions!

In [309]:
fb['counts'].head()

In [310]:
np.cumsum(fb['counts']).head()

In [347]:
preds = ar.predict()
full = fb['counts'].values[0] +  np.cumsum(preds)

f, a = plt.subplots()
a.plot(fb.index[1:15], fb[1:15], 'r', label='Data')
a.plot(fb.index[1:15], full[1:15], 'k', label='Fit')
plt.legend();

In [348]:
f, a = plt.subplots()
a.plot(fb.index[1:30], fb['counts'].diff()[1:30], label='Data')
a.plot(fb.index[1:30], preds[1:30], label='Fit')
plt.legend();

### Unemployment Data

In [349]:
data = pd.read_csv('data/seasonally-adjusted-quarterly-us.csv')

In [116]:
data.columns = ['year_q', 'unemp_rate']
data['unemp_rate'] = data['unemp_rate'].map(lambda x:\
                                            float(str(x).replace('%', '')))
data.dropna(inplace=True)

In [120]:
data['date'] = pd.to_datetime(data['year_q']).dt.to_period('Q')
data.set_index('date', inplace=True, drop=True)

In [314]:
data.head()

In [315]:
p=2
q=0
ar2 = ARMA(data['unemp_rate'].diff()[1:].values, (p, q)).fit()

In [316]:
preds2 = ar2.predict()
full2 = data['unemp_rate'].values[0] + np.cumsum(preds2)

f, a = plt.subplots()
a.plot(data.index.to_timestamp()[1:], data['unemp_rate'][1:], 'r', label='Data')
a.plot(data.index.to_timestamp()[1:], full2, 'k', label='Fit')
plt.legend();

In [317]:
f, a = plt.subplots()
a.plot(data.index.to_timestamp()[1:], data['unemp_rate'].diff()[1:], label='Data')
a.plot(data.index.to_timestamp()[1:], preds2, label='Fit')
plt.legend();